In [2]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.model_selection import train_test_split
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Download NLTK stop words
# nltk.download('stopwords')
# nltk.download('punkt')

# # Load your data
# data_transposed = pd.read_excel('indonesia_tourism_reviews.xlsx').transpose()
# review_columns = data_transposed.columns[:-1]

# # Advanced text preprocessing
# stop_words = set(stopwords.words('english'))

# def preprocess_text(text):
#     words = word_tokenize(text)
#     words = [w.lower() for w in words if w.isalpha()]  # Remove punctuation and numbers
#     words = [w for w in words if not w in stop_words]  # Remove stop words
#     return ' '.join(words)

# data_transposed['combined_reviews'] = data_transposed[review_columns].apply(lambda x: ' '.join(x.astype(str)), axis=1)
# data_transposed['combined_reviews'] = data_transposed['combined_reviews'].apply(preprocess_text)

# # Concatenate place names with reviews
# data_transposed['text_with_place'] = data_transposed.index + ' ' + data_transposed['combined_reviews']

# # Prepare the dataset for TensorFlow
# texts = data_transposed['text_with_place'].values
# labels = data_transposed.index

# # Tokenize and pad sequences
# tokenizer = Tokenizer(num_words=10000)
# tokenizer.fit_on_texts(texts)
# sequences = tokenizer.texts_to_sequences(texts)
# word_index = tokenizer.word_index
# data = pad_sequences(sequences, maxlen=200)

# # Load GloVe embeddings
# def load_glove_embeddings(glove_path, word_index, embedding_dim=100):
#     embeddings_index = {}
#     with open(glove_path, encoding='utf-8') as f:
#         for line in f:
#             values = line.split()
#             word = values[0]
#             coefs = np.asarray(values[1:], dtype='float32')
#             embeddings_index[word] = coefs

#     embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
#     for word, i in word_index.items():
#         embedding_vector = embeddings_index.get(word)
#         if embedding_vector is not None:
#             embedding_matrix[i] = embedding_vector

#     return embedding_matrix

# glove_path = 'glove.6B.100d.txt'  # Ensure you have the GloVe embeddings file
# embedding_dim = 100
# embedding_matrix = load_glove_embeddings(glove_path, word_index, embedding_dim)

# # Improved Model Architecture
# inputs = tf.keras.Input(shape=(200,))
# x = tf.keras.layers.Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)(inputs)
# x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
# x = tf.keras.layers.GlobalAveragePooling1D()(x)
# x = tf.keras.layers.Dense(128, activation='relu')(x)
# outputs = tf.keras.layers.Dense(embedding_dim)(x)

# model = tf.keras.Model(inputs, outputs)
# model.compile(optimizer='adam', loss='mse')

# # Train the Model
# text_embeddings = model.predict(data)

# similarity_scores_dict = {}

# review_df= pd.read_excel('indonesia_tourism_reviews.xlsx')

# # Recommendations
# def get_similarity_scores(selected_text_embedding, text_embeddings):
#     similarity_scores = cosine_similarity(selected_text_embedding.reshape(1, -1), text_embeddings)
#     return similarity_scores

# selected_place = 'Museum Pendidikan Nasional UPI'
# selected_place_index = labels.get_loc(selected_place)
# selected_place_embedding = text_embeddings[selected_place_index]

# similarity_scores = get_similarity_scores(selected_place_embedding, text_embeddings)

# sorted_similarities = sorted(list(enumerate(similarity_scores[0])), key=lambda x: x[1], reverse=True)

# # Display top 5 recommendations with their similarity percentages (excluding the selected place itself)
# top_recommendations_with_scores = [(labels[i[0]], i[1] * 100) for i in sorted_similarities[1:10]]
# print(top_recommendations_with_scores)

# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# # Enable Select TensorFlow ops
# converter.target_spec.supported_ops = [
#     tf.lite.OpsSet.TFLITE_BUILTINS,  # TFLite built-in ops.
#     tf.lite.OpsSet.SELECT_TF_OPS     # Select TensorFlow ops.
# ]

# # Disable experimental lowering of TensorList ops
# converter._experimental_lower_tensor_list_ops = False

# # Convert the model
# try:
#     tflite_model = converter.convert()
#     # Save the TFLite model
#     with open('model.tflite', 'wb') as f:
#         f.write(tflite_model)
#     print("Model conversion successful!")
# except Exception as e:
#     print("Model conversion failed:", e)
# # tflite_model = converter.convert()

# # Save the tokenizer
# import json

# tokenizer_json = tokenizer.to_json()
# with open('tokenizer.json', 'w') as f:
#     f.write(tokenizer_json)

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Download NLTK stop words
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
# Load your data
data = pd.read_excel('indonesia_tourism_reviews.xlsx')

In [ ]:
# Advanced text preprocessing
stop_words = set(stopwords.words('english'))

In [ ]:
def preprocess_text(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words if w.isalpha()]  # Remove punctuation and numbers
    words = [w for w in words if not w in stop_words]  # Remove stop words
    return ' '.join(words)

In [ ]:
# Combine reviews from the specified columns
review_columns = ['review1', 'review2', 'review3', 'review4', 'review5']
data['combined_reviews'] = data[review_columns].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
data['combined_reviews'] = data['combined_reviews'].apply(preprocess_text)

In [ ]:
# Concatenate place names with reviews
data['text_with_place'] = data['Place_Name'] + ' ' + data['combined_reviews']

In [ ]:
# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
data_sequences = pad_sequences(sequences, maxlen=200)

In [ ]:
# Load GloVe embeddings
def load_glove_embeddings(glove_path, word_index, embedding_dim=100):
    embeddings_index = {}
    with open(glove_path, encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [ ]:
glove_path = 'glove.6B.100d.txt'  # Ensure you have the GloVe embeddings file
embedding_dim = 100
embedding_matrix = load_glove_embeddings(glove_path, word_index, embedding_dim)

In [ ]:
# Improved Model Architecture
inputs = tf.keras.Input(shape=(200,))
x = tf.keras.layers.Embedding(len(word_index) + 1, embedding_dim, weights=[embedding_matrix], trainable=False)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(embedding_dim)(x)

In [ ]:
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='mse')

In [ ]:
# Train the Model
text_embeddings = model.predict(data_sequences)

In [1]:
# Calculate similarity scores for all destinations
similarity_matrix = cosine_similarity(text_embeddings)

# Create a DataFrame for the similarity matrix
similarity_df = pd.DataFrame(similarity_matrix, index=labels, columns=labels)

# Save the similarity matrix to an Excel file
similarity_df.to_excel('tourism_similarity_matrix.xlsx')

print("Similarity matrix saved to 'tourism_similarity_matrix.xlsx'")

NameError: name 'cosine_similarity' is not defined

In [2]:
import pandas as pd

# Baca file CSV
df = pd.read_csv('similarities_description_based.csv')

# Buat daftar unik dari Place dan Similar Place
places = list(set(df['Place']).union(set(df['Similar Place'])))

# Inisialisasi matriks dengan nilai 0
matrix = pd.DataFrame(0, index=places, columns=places)

# Isi matriks dengan nilai Score
for _, row in df.iterrows():
    matrix.at[row['Place'], row['Similar Place']] = row['Score']

# Simpan matriks ke dalam file Excel
matrix.to_excel('desc_matrix.xlsx', engine='openpyxl')

print("Matriks berhasil disimpan dalam file destination_matrix.xlsx")


Matriks berhasil disimpan dalam file destination_matrix.xlsx


In [ ]:
import pandas as pd
from transformers import pipeline

# Baca file Excel
df = pd.read_excel('indonesia_tourism.xlsx')

# Inisialisasi pipeline untuk analisis sentimen
sentiment_pipe = pipeline("text-classification", model="w11wo/indonesian-roberta-base-sentiment-classifier")

# Fungsi untuk melakukan analisis sentimen pada satu review
def analyze_sentiment(review):
    if pd.isna(review):  # Jika review adalah NaN, kembalikan None
        return None
    result = sentiment_pipe(review)
    return result[0]['label']

# Daftar kolom review
review_columns = [f'Review{i}' for i in range(1, 6)]

# Lakukan analisis sentimen pada setiap kolom Review1 hingga Review5
for review_col in review_columns:
    sentiment_col = f'Sentiment_{review_col}'
    df[sentiment_col] = df[review_col].apply(analyze_sentiment)

# Simpan hasil ke dalam file baru
df.to_excel('indonesia_tourism_with_sentiments.xlsx', index=False, engine='openpyxl')

print("Analisis sentimen selesai dan hasil disimpan dalam file indonesia_tourism_with_sentiments.xlsx")
